In [ ]:
%cd /mnt/md0/nfs/ryan/CleanCode/Projects/Peekaboo/Experiments/Dreams

In [ ]:
import rp
from IPython.display import HTML, clear_output
import source.peekaboo as peekaboo
from source.peekaboo import PeekabooResult

In [ ]:
class Result(PeekabooResult):
    @property
    def algorithm_name(self):
        #Specific to these datasets
        return self.name.split('.')[-1]

In [ ]:
def load_results(peekaboo_results_folder='./peekaboo_results'):
    # print(rp.get_absolute_path(peekaboo_results_folder))
    results=[]
    for result_collection in rp.get_subfolders(peekaboo_results_folder):
        # print("-----------",result_collection)
        for result_folder in rp.get_subfolders(result_collection):
            result=Result(result_folder)
            if result.is_valid:
                results.append(result)
            # else:
                # print("Invalid result at %s"%result_folder)
    return results

def cluster_results_by_algorithm(results)->dict:
    return rp.cluster_by_key(results, lambda x:x.algorithm_name, as_dict=True)

In [ ]:
def save_xiang_dir(peekaboo_results_folder='./peekaboo_results', xiang_dir_name='ref_voc'):
    results = load_results(peekaboo_results_folder)
    
    results_per_algorithm = cluster_results_by_algorithm(results)

    for algorithm_name, result_subset in results_per_algorithm.items():

        xiang_dir = rp.make_directory('untracked/for_xiang/%s/%s/alpha'%(xiang_dir_name, algorithm_name))
        print("Making",xiang_dir)

        display_eta=rp.eta(len(result_subset))
        for index, result in enumerate(result_subset):
            display_eta(index)
            original_path = result.alpha_path
            new_path = rp.joined_paths(xiang_dir, result.image_name)
            if not rp.is_image_file(new_path):
                rp.copy_file(original_path, new_path)

In [ ]:
rp.get_subfolders('/mnt/md0/nfs/ryan/CleanCode/Projects/Peekaboo/Experiments/Dreams/peekaboo_results/2008_008525.motorcycle.clip_raster_bilateral')

In [ ]:
#TODO: Fix 'tv/monitor' prompt! Bad! It subfoldered!
save_xiang_dir('peekaboo_results','ref_voc')
save_xiang_dir('peekaboo_results_ref_voc_mo','ref_voc_mo')
save_xiang_dir('peekaboo_results_coco_fixed','coco')

In [ ]:
%pwd

In [ ]:
peekaboo_results_folder='./peekaboo_results_coco_fixed'
# peekaboo_results_folder='./peekaboo_results'
results = load_results(peekaboo_results_folder)
results_per_algorithm = cluster_results_by_algorithm(results)
rp.ic(len(results),list(results_per_algorithm))
raster_bilateral = results_per_algorithm['raster_bilateral']
result_subset=raster_bilateral

In [ ]:
result_subset=raster_bilateral
# result_subset=rp.random_batch(result_subset,49)

In [ ]:
def outline_result(result):
    image=result.image
    threshold=.2

    alpha=result.alpha
    std=alpha.std()
    alpha=alpha-alpha.mean()*.55
    alpha=alpha/std/2
    
    image=rp.cv_resize_image(image,rp.get_image_dimensions(alpha))
    alpha=rp.cv_gauss_blur(alpha, sigma=3)
    alpha=rp.cv_dilate(alpha, diameter=5, circular=True)
    alpha=rp.as_float_image(alpha)
    mask=alpha>threshold

    def get_outline_from_mask(thickness):
        #Returns a mask for the outline
        return rp.cv_dilate(mask, diameter=thickness, circular=True) != rp.cv_erode(mask, diameter=thickness, circular=True)
    
    output=image
    output=rp.blend_images(output, (.0,.0,.0,.3), ~mask)
    output=rp.blend_images(output, (1,1,1,.3), mask)
    output=rp.blend_images(output, (0,0,0,1), get_outline_from_mask(7))
    output=rp.blend_images(output, (0,1,0,1), get_outline_from_mask(3))
    
    output=rp.horizontally_concatenated_images(alpha,output)
    
    return output

for result in result_subset:
    
    rp.display_image(outline_result(result))
        
    

In [ ]:
rp.display_image(rp.cv_dilate(rp.as_byte_image(rp.as_grayscale_image(result_subset[0].alpha)),circular=True))

In [ ]:
rp.cv_dilate(rp.as_byte_image(result_subset[0].image),circular=True)